In [2]:
import pandas as pd
import numpy as np


In [3]:
mp_file_path = '/Users/apple/Downloads/nfl/mediaperformance.csv'
nfl_file_path = '/Users/apple/Downloads/nfl/nfladverts.csv'

mp_data = pd.read_csv(mp_file_path)
nfl_data = pd.read_csv(nfl_file_path)


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 数据清洗

In [4]:
# 重新命名列

nfl_data.columns = ['transaction_id', 'customer_id', 'date', 'nflweek', 'platform', 'market',
       'audience', 'segment', 'gender', 'age_group', 'tickets',
       'shop', 'fantasy', 'new_to_database', 'email_opt_in',
       'favourite_team', 'sku', 'buy_type', 'converted_free_trial',
       'revenue_usd']

# nfl_data['revenue_usd'].describe()
# mp_data.columns

### 重复值

In [5]:
mp_unique = mp_data.groupby(['date', 'nflweek', 'numweek', 'platform', 'market', 'audience']).size().reset_index(name='Freq')
mp_unique = mp_unique.sort_values(by=['Freq'], ascending=False)
# print(mp_unique)

nfl_data.head()
nfl_unique = nfl_data.groupby(['transaction_id']).size().reset_index(name='Freq')
nfl_unique = nfl_unique.sort_values(by=['Freq'], ascending=False)
# print(nfl_unique)
# data.drop_duplicates(subset ="columns Name", keep = False, inplace = True) 


### 缺失值

In [6]:
mp_null_total = mp_data.isnull().sum(axis=0).sort_values(ascending=False)
mp_null_percent = (mp_data.isnull().sum()/len(mp_data.index)).sort_values(ascending=False).round(3)
mp_missing_data = pd.concat([mp_null_total, mp_null_percent], axis=1, keys=['Total', 'Percent'])

nfl_null_total = nfl_data.isnull().sum(axis=0).sort_values(ascending=False)
nfl_null_percent = (nfl_data.isnull().sum()/len(nfl_data.index)).sort_values(ascending=False).round(3)
nfl_missing_data = pd.concat([nfl_null_total, nfl_null_percent], axis=1, keys=['Total', 'Percent'])


# 去掉没有transaction_id的行
nfl_data = nfl_data.dropna(subset=['transaction_id'])

# 没有填喜好球队的编为‘No Team’
nfl_data['favourite_team'] = nfl_data['favourite_team'].fillna('No Team')


In [7]:
# 没填性别编为'U'
nfl_data['gender'] = nfl_data['gender'].fillna('U')

# 由于gender缺失太多，和其他列也没有存在明显的逻辑关系，尝试看有没有男女id差异，结果没有
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html
gender_predict = nfl_data[nfl_data.gender.notnull()]
gender_pivot = pd.pivot_table(gender_predict, values='customer_id', index=['gender'], aggfunc=np.mean)

# 按照逻辑填充buy_type
# nfl_data['buy_type'].unique()
nfl_data['buy_type'] = nfl_data.apply(
    lambda row: 
        'Buy Now' if row['sku'] =='Free' else 
            ('Buy Now' if (row['converted_free_trial'] == 0 and row['revenue_usd'] > 0) else 
                 ('Free Trial' if (row['converted_free_trial'] == 0 and row['revenue_usd'] == 0) or (row['converted_free_trial'] == 1 and row['revenue_usd'] > 0) else 
                      row['buy_type'])), axis=1)

# nfl_data.isnull().sum(axis=0).sort_values(ascending=False)


### 异常值

In [8]:
# binary都变成0&1，去掉异常值
# nfl_data['market'].unique()
nfl_data['tickets'] = np.where(nfl_data['tickets'] == 'N', 0, 1)
nfl_data['shop'] = np.where(nfl_data['shop'] == 'N', 0, 1)
nfl_data['fantasy'] = np.where(nfl_data['fantasy'] == False, 0, 1)
nfl_data['new_to_database'] = np.where(nfl_data['new_to_database'] == 'N', 0, 1)
nfl_data['email_opt_in'] = np.where(nfl_data['email_opt_in'] == 'False', 0, np.where(nfl_data['email_opt_in'] == '0', 0, 1))
nfl_data['buy_type'] = np.where(nfl_data['buy_type'] == 'Free Trial', 0, 1)                                   
nfl_data['favourite_team'] = np.where(nfl_data['favourite_team'] == 'NFL', 'No Team', nfl_data['favourite_team'])
nfl_data['favourite_team'] = np.where(nfl_data['favourite_team'] == 'No Team', 0, 1)


In [9]:
# 纠正sku、buy_type、converted_free_trial的逻辑错误
# https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o
# https://discuss.analyticsvidhya.com/t/how-to-resolve-python-error-cannot-compare-a-dtyped-int64-array-with-a-scalar-of-type-bool/73065
nfl_data['sku'] = np.where((nfl_data['revenue_usd'] == 0) & (nfl_data['sku'] != 'Pro'), 'Free', nfl_data['sku'])
nfl_data['sku'] = np.where((nfl_data['revenue_usd'] > 0) & (nfl_data['sku'] == 'Free'), 'Pro', nfl_data['sku'])
nfl_data['buy_type'] = np.where((nfl_data['sku'] == 'Pro') & (nfl_data['revenue_usd'] == 0), 0, nfl_data['buy_type'])
nfl_data['converted_free_trial'] = np.where((nfl_data['sku'] == 'Pro') & (nfl_data['buy_type'] == 0) & (nfl_data['revenue_usd'] > 0), 1, nfl_data['converted_free_trial'])
nfl_data['converted_free_trial'] = np.where((nfl_data['sku'] == 'Pro') & (nfl_data['buy_type'] == 1) & (nfl_data['revenue_usd'] > 0), 0, nfl_data['converted_free_trial'])


In [10]:
mp_data.to_excel(r'/Users/apple/Downloads/nfl/mp_data.xlsx', index=True)

In [11]:
nfl_data.to_csv(r'/Users/apple/Downloads/nfl/nfl_data.csv', index=True)